1. 신경망을 구성하는 구성 요소에 대한 심화 학습
2. 딥러닝 아키텍처를 구축하기 위한 파이토치의 상위 수준 기능 소개
3. 실세계 이미지 분류 문지ㅔ에 딥러닝 알고리즘 적용

In [1]:
#파이토치가 제공하는 선형레이어는 강력하다!
#그 이유는 코드 한 줄로 선형 레이어 구성이 가능하기 때문
import torch
from torch.nn import Linear

myLayer = Linear(in_features=10, out_features=5, bias=True)

In [2]:
from torch.autograd import Variable
inp = Variable(torch.randn(1,10))
myLayer(inp)

tensor([[-0.1365, -0.1122,  0.5193, -0.0531, -0.7838]],
       grad_fn=<AddmmBackward>)

In [3]:
myLayer.weight

Parameter containing:
tensor([[ 0.1234,  0.0045, -0.3113, -0.1605,  0.1625,  0.1175, -0.2891,  0.1061,
         -0.1082, -0.0251],
        [ 0.1526, -0.2140, -0.1315, -0.1597, -0.1612,  0.2052,  0.2502, -0.0885,
         -0.0009,  0.1483],
        [-0.1336, -0.2059,  0.0715,  0.0730,  0.3071, -0.2131, -0.2360, -0.2783,
          0.1445,  0.0291],
        [ 0.0520,  0.0235,  0.1594,  0.1464,  0.1206, -0.2460, -0.0932,  0.1992,
          0.1152, -0.1846],
        [-0.2983,  0.2996, -0.1959, -0.1194, -0.2300,  0.2146,  0.3082,  0.0307,
         -0.1074, -0.2180]], requires_grad=True)

In [4]:
myLayer.bias

Parameter containing:
tensor([ 0.3157, -0.1597, -0.2168, -0.1422, -0.0414], requires_grad=True)

선형 레이어는 dense layer 혹은 fully connected layer이라고 불리기도 한다.

In [5]:
# 복수 레이어 구성하는 방법 
# 첫번째, 다음 레이어에 이전 레이어의 출력을 입력한다.

myLayer1 = Linear(10,5)
myLayer2 = Linear(5,2)
myLayer2(myLayer1(inp))

# 하지만 이와같은 방법은 하나의 레이어를 갖는 아키텍처로 표현될 수 있기 때문에 학습하는데 전혀 도움이 되지 않는다...
# 이 한계를 극복하기 위해 비선형 함수를 활용한다.

tensor([[-0.1349,  0.0269]], grad_fn=<AddmmBackward>)

**비선형 활성 함수**
1. *시그모이드* : 최근엔 잘 활용되지 않음. 함수의 출력값이 0또는 1에 가까워지면 기우리가 0으로 수렴하게 되면서, 가중치가 조정되지 않는 죽은 뉴런이 발생한다는 치명적인 단점이 있다.
2. *Tanh *: 시그모이드 보다 많이 사용된다. -1부터 1까지 값으로 변환한다. 시그모이드와 동일한 단점이 존재는하지만, 시그모이드보다 범위가 더 넓어 시그모이드에 비해 단점이 적게 발생한다. 
3. *ReLU* : 최근 가낭 큰 인기를 얻고있는 빈성형 활성함수. 음수는 0을 양수는입력값 그대로를 반환한다. 따라서 가중치를 더 빨리 찾도록 하며, 시그모이드와 Tanh에 비해 연산이 없다. 하지만 역전파를 진행하면서 기울기가 커지면 뉴런이 죽어버리기도 한다 (하지만 이 말이 무슨 말인지는 아직 모르겠다.)
4. *leaky ReLU* : ReLU의 변형된 형태. 음수가 0이 아닌 0에 가까운 작은 수로 반환된다. 하지만 아직 일관된 우수 성능을 보이진 않는다.

In [6]:
#ReLU
from torch.nn import ReLU
import torch
sample_data = torch.Tensor([[1,2,-1,-1]])
myRelu = ReLU()
myRelu(sample_data)

tensor([[1., 2., 0., 0.]])

파이터치 딥러닝 알고리즘
파이토치에서는 모든 네트워크가 클래스 형태로 개발된다.
nn.Module 클래스를 상속하고 __init__와 forward 메서드를 구현해야한다.
__init__메서드에서는 앞에서 다룬선형 레이어와 같은 레이어를 초기화, 
forward 메서드에서는 __init__메서드에서 초기화한 레이어에 입력 데이터를 전달하고 최종 출력을 반호나.
비선형 함수는 때때로  forward 에서 직접 사용될 수도 있고, __init__메서드에서도 사용된다.

In [7]:
# 예시

from torch import nn as nn

class MyFirstNetwork(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(MyFirstNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        out = self.layer1(input)
        out = nn.ReLU(out)
        out = self.layer2(out)
        return out

보통 머신 러닝 문제를 해결하기 위해서는, 문제 유형에 따라 마지막 레이어의 유형이 바뀐다.
대표적인 마지막 레이어는

1. **회귀** : 예측 회귀문제의 경우 마지막 레이어로 1개의 숫자를 출력하는 **선형 레이어** 사용
2. **이진분류** : 마지막 레이어로 **시그모이드 활성화 함수**를 사용
3. **다중분류** : **소프트맥스 레이어**(다중 분류 확률 계산)가 사용됨

**오차함수**

회귀 문제에서 주로 사용되는 오차함수는 **MSE**(평균 제곱 오차)를 사용

In [8]:
# 예제

loss = nn.MSELoss()
input = Variable(torch.randn(3,5), requires_grad = True)
target = Variable(torch.randn(3,5))
output = loss(input, target)
output.backward()

분류에서는 오차함수로 교차 엔트로피 오차(cross entryopy loss)를 사용

이때 교차 엔트로피 오차란 간단히 말하면 예측이 정확하지 않으면 오차가 높아지고 예측이 정확할수록 오차가 낮아지는 그런 계산법.

In [9]:
def cross_entropy(true_label, prediction):
    if true_label == 1:
        return -log(prediction)
    else:
        return -log(1-prediction)

In [10]:
#예제
loss = nn.CrossEntropyLoss()
input = Variable(torch.randn(3,5), requires_grad = True)
target = Variable(torch.LongTensor(3).random_(5))
output = loss(input, target)
output.backward()

pytorch에서 제공하는 오차함수
1. **L1 Loss** : 정규화에서 주로 사용
2. **MSE Loss** : 회귀 문제에서 주로 사용
3. **Cross-entropy Loss** : 이진분류, 다중분류
4. **NLL Loss** :  분류 문제에서 특정 가중치를 사용해 데이터셋 불균형을 처리할 때 사용
5. **NLL Loss2d** : 주로 이미지 분할과 관련된 문제에서 픽셀 단위 분류에 사용

# 네트워크 아키텍처 최적화
오차를 줄이고 알고리즘의 정확도를 높이기 위해 가중치를 최적화.

옵티마이저 작동 방식을 알고싶다면

http://colah.github.io/posts/2015-08-Backprop/

http://ruder.io/deep-learning-optimization-2017/

# 딥러닝을 이용한 이미지 분류

In [17]:
#kaggle 파일이 폼이 조금 달라서 수정.

In [37]:
from glob import glob
import os

path = "C:/Users/user/Desktop/2021-1/python/dogsandcats/data"

In [38]:
files = glob(os.path.join(path, '*.jpg'))

In [39]:
no_of_images = len(files)

In [41]:
import numpy as np

shuffle = np.random.permutation(no_of_images)

#검증 이미지를 저장할 검증용 디렉터리 생성
os.mkdir(os.path.join(path, 'valid'))
os.mkdir(os.path.join(path,'train'))

for t in ['train', 'valid']:
    for folder in ['dog', 'cat']:
        os.mkdir(os.path.join(path, t, folder))



In [ ]:
#valid 폴더에 이미지 2000장 복사
for i in shuffle[:2000]:
    folder = files[i].split('\\')[-1].split('.')[0]
    image = files[i].split('\\')[-1]
    os.rename(files[i], os.path.join(path,'valid', folder, image))
    
#train 폴더에 나머지 이미지 복사
for i in shuffle[2000:]:
    folder = files[i].split('\\')[-1].split('.')[0]
    image = files[i].split('\\')[-1]
    os.rename(files[i], os.path.join(path,'train', folder, image))

In [4]:
#ImageFolder 클래스를 사용해 변환을 적용하고 이미지를 로딩한다.
from torchvision import transforms
from torchvision.datasets import ImageFolder

simple_transform = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
train = ImageFolder('C:/Users/user/Desktop/2021-1/python/dogsandcats/data/train/', simple_transform)
valid = ImageFolder('C:/Users/user/Desktop/2021-1/python/dogsandcats/data/valid/', simple_transform)

train 객체는 데이터셋의 모든 이미지와 관련 레이블ㅇ르 보관한다. 
class_to_idx는 데이터셋에서 사용하는 각 분류 클래스와 대응되는 인덱스 정보, 
train 객체의 classes속성은 분류 클래스 목록 제공

In [5]:
def imshow(inp):
    """Tensor를 이미지로 출력"""
    inp = inp.numpy().transpose((1,2,0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp,0,1)
    plt.imshow(inp)

In [9]:
import torch

train_data_gen = torch.utils.data.DataLoader(train, batch_size = 64, num_workers=3)
valid_data_gen = torch.utils.data.DataLoader(valid, batch_size = 64, num_workers=3)

In [10]:
from torchvision import models
model_ft = models.resnet18(pretrained = True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = torch.nn.Linear(num_ftrs,2)

위에서 쓰인 torchvision.models.resnet18은 ResNet 알고리즘의 인스턴스를 생성한다. 

위 코드에서 model_ft는 ResNet 인스턴스이다.

ResNet 아키텍처는 Conv2d, BatchNorm2d, MaxPool2d 등 특수 형태로 결합된 여러 레이어로 구성되어 있음

알고리즘 클래스는 pretrained 인자를 가저 True로 설정될 시 알고리즘은 자동으로 1000가지 카테고리를 예측하기 위해 특정 ImageNet분류 문제를 미리 학습한 가중치를 설정한다. 

ResNet 알고리즘은 1000개의 카테고리 중 하나를 예측하도록 훈련되어 있지만 예제는 개와 고양이 2개 카테고리 중 하나를 예측해야 하므로 ResNet 알고리즘을 수정해야한다.

model_ft.fc = torch.nn.Linear(num_tfrs, 2) 

를 통해 선형 레이어인 ResNet 모델의 가장 마지막 레이어를 가져와 출력 피처를 2로 변경한다. 

# 모델학습
모델을 학습시키기 위해서는 오차 함수와 옵티마이저가 필요

In [14]:
# 오차 함수와 옵티마이저
learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss() #오차함수
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr = 0.001, momentum = 0.9) #SGD 기반 옵티마이저
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1) # StepLR함수는 학습률을 동적으로 변경해준다.

In [15]:
#train_model는 모델을 가져와 여러 에폭을 실행하고, 오차를 줄임으로써 알고리즘의 가중치를 조정해준다!

def train_model(model, criterion, optimizer, scheduler, num_epochs = 25):
    since = time.time()
    
    best_model_wts = model.state_dict()
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        #각 에폭은 학습과 검증 단계로 구성
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True) # 학습 모드로 모델 설정
            else:
                model.train(False) # 평가 모드로 모델 설정
            
            running_loss = 0.0
            running_corrects = 0
            
            #데이터 반복
            for data in dataloaders[phase]:
                #데이터 획득
                inpus, labels = data
                if is_cuda:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    input, labels = Variable(inputs), Variable(labels)
                
                # 파라미터 기울기 최적화
                optimizer.zero_grad()
                
                #forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outpus, labels)
                
                #학습단계에서만 수행 (backward + optimize)
                if phase == "train":
                    loss.backward()
                    optimizer.step()
                
                # 통계 
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format( phase, epoch_loss, epoch_acc))
            
            #모델 복사 (deep copy)
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
                
        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    #최적 가중치 로드
    model.load_state_dict(best_model_wts)
    return model

아직 저 함수를 완벽하게 이해하긴 힘들지만 책의 말에 의하면 위 함수는 
1. 모델에 이미지를 전달하고 오차를 계산
2. 학습 단계에서 역전파(backward)를 수행, 유효성 검사/테스트 단계에서는 가중치를 업데이트 하지 않음
3. 에폭 동안 각 배치에서 발생하는 오차를 누적
4. 최상의 모델이 저장되고 점증 정확도를 인쇄
의 방식으로 동작한다고 함.